# NAPE Calcium Imaging Preprocessing Pipeline

Finds any .tif, .tiff, .h5 files in the requested directory and performs SIMA-based motion correction and fft-based bidirection 
offset correction. This code parallelizes the computation at the session level by passing the multiple file paths (if there are 
more than one recordings) to the multiprocessing map function. The script sima_motion_correction contains the wrapping
code for SIMA and the custom-created class for bidirection offset correction.

Two simple ways to execute this in command line:  
A) sima_motion_correct_batch; then input the path to the directory containing your raw files

B) sima_motion_correct_batch.batch_process(path_to_directory)

__Otherwise, in this jupyter notebook, just run all cells in order (shift + enter)__

See these documentations for details
------------------------------------

https://github.com/losonczylab/sima
http://www.losonczylab.org/sima/1.3.2/
https://www.frontiersin.org/articles/10.3389/fninf.2014.00080/full

Required Packages
-----------------
sima, glob, multiprocessing, numpy, h5py, pickle (optional if want to save displacement file) 

Custom code requirements: sima_motion_correction, bidi_offset_correction

Parameters
----------

fdir : string
    root file directory containing the raw tif, tiff, h5 files. Note: leave off the last backslash.

Optional Parameters
-------------------

max_disp : list of two entries
    Each entry is an int. First entry is the y-axis maximum allowed displacement, second is the x-axis max allowed displacement.
    The number of pixel shift for each line cannot go above these values.
    Note: 50 pixels is approximately 10% of the FOV (512x512 pixels)
    
    Defaults to [30, 50]
    
save_displacement : bool 
    Whether or not to have SIMA save the calculated displacements over time. def: False; NOTE: if this is switched to True,
    it can double the time to perform motion correction.
    
    Defaults to False
    
Output
-------
motion corrected file (in the format of h5) with "\_sima_mc" appended to the end of the file name


In [ ]:
import sima_motion_bidi_correction 
import calculate_neuropil
import glob
from fnmatch import fnmatch
import multiprocessing as mp
import os
import single_file_process
import files_to_analyze

In [ ]:
def batch_process(root_dir, max_disp = [30, 50], save_displacement = False):
    
    if not root_dir: # if string is empty, load predefined list of files in files_to_analyze
        
        fparams = files_to_analyze.define_fparams()
        
    else:
        
        root_dir = root_dir + '\\'

        # declare initialize variables to do with finding files to analyze
        fparams = []
        fpaths = []
        types = ['*.tif', '*.tiff', '*.h5']
        exclude_strs = ['spatialweights', '_sima_mc', '_trim_dims']

        # find files to analyze
        for path, subdirs, files in os.walk(root_dir): # os.walk grabs all paths and files in subdirectories
            for name in files:
                # make sure file of any image file, \ continues the if statement to next line
                if any([fnmatch(name, ext) for ext in types]) and not any([exclude_str in name for exclude_str in exclude_strs]): # but don't include processed files
                    tmp_dict = {}
                    tmp_dict['fname'] = name
                    tmp_dict['fdir'] = path
                    tmp_dict['max_disp'] = max_disp
                    tmp_dict['save_displacement'] = save_displacement

                    fparams.append(tmp_dict)
                    
    # print info to console
    num_files = len(fparams)
    if num_files == 0:
        raise Exception("No files to analyze!")
    print(str(num_files) + ' files to analyze')
    
    # determine number of cores to use and initialize parallel pool
    num_processes = min(mp.cpu_count(), num_files)
    print('Total CPU cores for parallel processing: ' + str(num_processes))
    pool = mp.Pool(processes=num_processes)
    
    # perform parallel processing; pass iterable list of file paths to the motion correction script
    pool.map(single_file_process.process, fparams)
    # single_file_process.process(fparams[0])

    pool.close()
    pool.join()

In [ ]:
%%time
if __name__ == "__main__":
    fdir = raw_input(r"Input root directory of tif, tiff, h5 files to analyze; note: Use FORWARD SLASHES to separate folder and leave the last backlash off!!  ")
    batch_process(fdir)
